**tools**

https://python.langchain.com/v0.1/docs/modules/tools/

multiple data sources

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)   #tool1

In [22]:
wiki.name

'wikipedia'

In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings #OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter  #to dive the documents into chunk

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings(model = "llama3.2:1b"))
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000243A6EEBCE0>, search_kwargs={})

In [ ]:
from langchain.tools.retriever  import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about Langchain you must use this tool.")
#tool2

In [18]:
retriever_tool.name

'langsmith_search'

In [ ]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name
#tool3

'arxiv'

Combining all the tools

In [42]:
tools = [wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\JN\\LANGCHAIN\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about Langchain you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002439F5D9760>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstore

**Agents**

https://python.langchain.com/v0.1/docs/modules/agents/

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

In [ ]:
#if you have open ai api key
from dotenv import load_dotenv
load_dotenv()
import os

os.environ['OPENAI_API_KEY'] = os.getenv('OPEN_API_KEY')
from langchain_openai import OpenAI

llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature = 0)

In [29]:
#else
from langchain.llms import Ollama
llm = Ollama(model = "llama3.2:1b", temperature = 0)

C:\Users\697sm\AppData\Local\Temp\ipykernel_7208\3731487512.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model = "llama3.2:1b", temperature = 0)


temperature = 0: The model will output the most probable response, making it more focused and repetitive.

temperature = 1: It balances creativity and accuracy.

temperature > 1: It may produce highly creative but less predictable responses, sometimes sacrificing coherence.

In [34]:
#for prompts we can create prompt by using chatprompt template
#but langhain has a library where prompts are already created

from langchain import hub
#get the  prompt from the library-you can modify this!
prompt = hub.pull("pisles/rag-prompt-llama-ps") #owner and path
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="[INST]<<SYS>> You are an assistant for analyzing transcriptions from body-worn cameras, interviews, surveillance records, and other official content. Use the retrieved context to provide analysis for officials working with federal and state agencies, such as police departments or investigators. Answer concisely within three sentences. If the answer is unclear, state that you don't know.<</SYS>> \nQuestion: {question} \nContext: {context} \nAnswer: [/INST]"), additional_kwargs={})]

In [ ]:
#agents

#if you have openai API key
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

ValueError: Prompt missing required variables: {'agent_scratchpad'}

In [48]:
#if you are using Ollama #generated from chat gpt

from langchain.agents import initialize_agent, AgentType

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION
)


In [49]:
## Agent Executer - To get the result from agents

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose = True) #verbose shows detials
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=AgentExecutor(verbose=False, tags=['chat-zero-shot-react-description'], agent=ChatAgent(llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv: A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query

In [50]:
agent_executor.invoke({"input" : "tell me about langsmith"})



> Entering new AgentExecutor chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'